In [2]:
import pandas as pd
import time
import requests
import json

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
# options.add_argument("headless")
s = Service(ChromeDriverManager().install())
chrome = webdriver.Chrome(options = options,service = s)
chrome1 = webdriver.Chrome(options = options,service = s)



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Driver [/home/cosbi/.wdm/drivers/chromedriver/linux64/106.0.5249.61/chromedriver] found in cache


In [3]:
chrome.get("https://sjmain.esunsec.com.tw/z/zh/zha/zha.djhtm")

In [28]:
table = chrome.find_element(by = By.ID, value = "oMainTable")
tbody = table.find_element(by = By.TAG_NAME, value = "tbody")

In [13]:
class_dict = {}

for idx in range(2, len(tbody), 1):
    a = td_t3t1[idx].find_element(by = By.TAG_NAME, value = "a")
    print(a.get_attribute("innerHTML"))
    chrome1.get(a.get_attribute("href"))
    table1 = chrome1.find_element(by = By.ID, value = "oMainTable")
    a1 = table1.find_elements(by = By.TAG_NAME, value = "a")
    # print([industry.get_attribute("innerHTML") for industry in a1])
    # class_dict[a.get_attribute("innerHTML")] = [industry.get_attribute("innerHTML") for industry in a1]
    # print(class_dict)

方便麵
製糖
非酒精飲料
PVC
AN
成衣
尼龍製品


KeyboardInterrupt: 